In [13]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

In [14]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
def qtd_eventos(integer):
    return 45 - integer

qtd_eventos = list(map(qtd_eventos, df.isnull().sum(axis=1).tolist()))
df['qtd_eventos'] = qtd_eventos
df = df[df.qtd_eventos >= 30]
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
df

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result,qtd_eventos
2,36866,ESPORTSTMNT02_2557390,19,2,20,24,2,4,20,3,...,0.80,4.8,1,0.00,0.8,5,0.60,3.2,0,31
7,36889,ESPORTSTMNT02_2557053,19,1,2,5,1,20,21,4,...,0.00,0.0,1,0.00,0.4,0,0.00,0.0,1,32
11,36894,ESPORTSTMNT02_2577994,0,20,2,1,5,3,6,20,...,0.00,0.0,1,1.00,12.0,13,0.46,1.9,1,30
33,36926,ESPORTSTMNT05_2530903,19,21,20,22,23,20,21,25,...,0.00,0.0,4,0.25,2.8,0,0.00,0.0,0,34
35,36928,ESPORTSTMNT05_2530937,0,20,21,1,22,3,24,2,...,0.33,6.8,2,0.00,1.8,1,0.00,8.0,1,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4485,40324,ESPORTSTMNT02_2840001,19,20,2,4,20,5,2,24,...,0.00,0.0,0,0.00,0.0,0,0.00,0.0,0,30
4528,40920,ESPORTSTMNT06_2460444,19,2,20,20,5,2,3,4,...,0.00,1.7,1,1.00,6.5,0,0.00,0.0,1,32
4548,40432,ESPORTSTMNT06_2420060,0,2,20,3,1,6,5,21,...,0.33,5.0,8,0.38,3.8,0,0.00,0.0,0,32
4551,40434,ESPORTSTMNT06_2420117,19,20,21,20,5,21,22,23,...,0.50,6.8,0,0.00,0.0,1,0.00,0.6,1,30


In [15]:
y = df['result'].copy()
X = df.drop(['golId','result','game','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
X

,event1,event2,event3,event4,event5,event6,event7,event8,event9,event10,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,qtd_eventos
2,19,2,20,24,2,4,20,3,6,1,...,5,0.80,4.8,1,0.00,0.8,5,0.60,3.2,31
7,19,1,2,5,1,20,21,4,1,3,...,0,0.00,0.0,1,0.00,0.4,0,0.00,0.0,32
11,0,20,2,1,5,3,6,20,2,4,...,0,0.00,0.0,1,1.00,12.0,13,0.46,1.9,30
33,19,21,20,22,23,20,21,25,5,24,...,0,0.00,0.0,4,0.25,2.8,0,0.00,0.0,34
35,0,20,21,1,22,3,24,2,4,7,...,3,0.33,6.8,2,0.00,1.8,1,0.00,8.0,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4485,19,20,2,4,20,5,2,24,3,20,...,0,0.00,0.0,0,0.00,0.0,0,0.00,0.0,30
4528,19,2,20,20,5,2,3,4,20,20,...,1,0.00,1.7,1,1.00,6.5,0,0.00,0.0,32
4548,0,2,20,3,1,6,5,21,22,1,...,3,0.33,5.0,8,0.38,3.8,0,0.00,0.0,32
4551,19,20,21,20,5,21,22,23,1,26,...,4,0.50,6.8,0,0.00,0.0,1,0.00,0.6,30


In [16]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(random_state=42,max_iter=10000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(random_state=42,max_iter=10000),
    'Support Vector Machine (RBF Kernel)': SVC(random_state=42,C=1,gamma=0.01,kernel='sigmoid',max_iter=50000),
    'Decission Tree': DecisionTreeClassifier(random_state=42),
    'Adaboost': AdaBoostClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting Classifier': GradientBoostingClassifier(random_state=42)
}

for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained
Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [18]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Balanced Accuracy': balanced_accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test)),
    'ROC/AUC Score': roc_auc_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [19]:
scores

,Model,Balanced Accuracy,Precision,Recall,F1-Score,ROC/AUC Score
0,Logistic Regression,0.838466,0.785714,0.868421,0.825000,0.838466
1,Support Vector Machine (Linear Kernel),0.862262,0.809524,0.894737,0.850000,0.862262
2,Support Vector Machine (RBF Kernel),0.931131,0.900000,0.947368,0.923077,0.931131
3,Decission Tree,0.867861,0.864865,0.842105,0.853333,0.867861
4,Adaboost,0.788354,0.750000,0.789474,0.769231,0.788354
5,Random Forest,0.917973,0.897436,0.921053,0.909091,0.917973
6,Gradient Boosting Classifier,0.915454,0.918919,0.894737,0.906667,0.915454


In [20]:
import pickle

with open("../models/model-30.pkl", "wb") as f:
    pickle.dump(models['Random Forest'], f)